In [1]:
!pip3.7 install -U Flask -t /home/sbc_dev/.local/lib/python3.7/site-packages --force-reinstall

  Using cached Flask-2.0.1-py3-none-any.whl (94 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached Werkzeug-2.0.1-py3-none-any.whl (288 kB)
  Using cached itsdangerous-2.0.1-py3-none-any.whl (18 kB)
  Using cached Jinja2-3.0.1-py3-none-any.whl (133 kB)
  Using cached importlib_metadata-4.6.3-py3-none-any.whl (17 kB)
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux2010_x86_64.whl (31 kB)
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)
  Using cached zipp-3.5.0-py3-none-any.whl (5.7 kB)
ERROR: uvicorn 0.11.1 has requirement click==7.*, but you'll have click 8.0.1 which is incompatible.
ERROR: ipykernel 6.0.2 has requirement importlib-metadata<4; python_version < "3.8.0", but you'll have importlib-metadata 4.6.3 which is incompatible.
ERROR: deeppavlov 0.8.0 has requirement numpy==1.18.0, but you'll have numpy 1.21.0 which is incompatible.
ERROR: deeppavlov 0.8.0 has requirement pandas==0.25.3, but you'll have pandas 1.3.0 which is incompatible.


In [1]:
!pip3.7 install langdetect --user

    100% |████████████████████████████████| 983kB 2.3MB/s ta 0:00:011
  Stored in directory: /Users/a17194839/Library/Caches/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect
You are using pip version 19.0.3, however version 21.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [288]:
import sys
sys.path.insert(0,'/Users/a17194839/Library/Python/3.7/lib/python/site-packages')

In [289]:
import re
import string
from collections import Counter, defaultdict
from typing import Dict, List, Tuple, Union, Callable

from itertools import permutations

import nltk
import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('punkt')

import numpy as np
import math
import pandas as pd
import torch
import torch.nn.functional as F

import faiss
from langdetect import detect, DetectorFactory

import json
import pickle
from flask import Flask, request, jsonify

In [290]:
app = Flask(__name__)

In [291]:
glue_qqp_dir = './data/QQP'
EMB_PATH_GLOVE = './data/glove.6B.50d.txt'
EMB_PATH_KNRM = './pkl/pretr_emb_krnm_50'
VOCAB_PATH = './pkl/vocab'

In [197]:
DetectorFactory.seed = 0
detect("Why do people still believe in flat earth?")

'en'

In [292]:
class GaussianKernel(torch.nn.Module):
    def __init__(self, mu: float = 1., sigma: float = 1., requires_grad = False):
        super().__init__()
        mu_ = np.array(mu)
        sigma_ = np.array(sigma)
        self.requires_grad = requires_grad
        self.mu = torch.nn.Parameter(torch.Tensor(mu_), requires_grad=self.requires_grad)
        self.sigma = torch.nn.Parameter(torch.Tensor(sigma_), requires_grad=self.requires_grad)

    def forward(self, x):
        adj = x - self.mu
        return torch.exp(-0.5 * adj * adj / self.sigma / self.sigma)        

        
class KNRM(torch.nn.Module):
    def __init__(self, embedding_matrix: np.ndarray, freeze_embeddings: bool = False, kernel_num: int = 30,
                 sigma: float = 0.1, exact_sigma: float = 0.001,
                 out_layers: List[int] = [15, 7]):
        super().__init__()
        self.embeddings = torch.nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix),
            freeze=freeze_embeddings,
            padding_idx=0
        )

        self.kernel_num = kernel_num
        self.sigma = sigma
        self.exact_sigma = exact_sigma
        self.out_layers = out_layers

        self.kernels = self._get_kernels_layers()

        self.mlp = self._get_mlp()

        self.out_activation = torch.nn.Sigmoid()

    def _get_kernels_layers(self) -> torch.nn.ModuleList:

        mus = [1.0]
        if self.kernel_num > 1:
            bin_size = 2.0 / (self.kernel_num - 1)  
            mus.append(1 - bin_size / 2)
            for i in range(1, self.kernel_num - 1):
                mus.append(mus[i] - bin_size)
        mus = list(reversed(mus))
        sigmas = [self.sigma] * (self.kernel_num - 1) + [self.exact_sigma]  
        
        gausskern_lst = [(GaussianKernel(mu,sigma)) for mu, sigma in zip(mus, sigmas)]
        kernels = torch.nn.ModuleList(gausskern_lst)
        return kernels

    def _get_mlp(self) -> torch.nn.Sequential:        
        if self.out_layers:
            output = []
            hidden_sizes = [self.kernel_num] + self.out_layers + [1]
            for i, hidden in enumerate(hidden_sizes[1:],1):
                output.append(torch.nn.ReLU())
                output.append(torch.nn.Linear(hidden_sizes[i-1], hidden))
        else:
            output = [torch.nn.Linear(self.kernel_num, 1)]
        return torch.nn.Sequential(*output)

    def forward(self, input_1: Dict[str, torch.Tensor], input_2: Dict[str, torch.Tensor]) -> torch.FloatTensor:
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)

        logits_diff = logits_1 - logits_2

        out = self.out_activation(logits_diff)
        return out

    def _get_matching_matrix(self, query: torch.Tensor, doc: torch.Tensor) -> torch.FloatTensor:
        query = self.embeddings(query)
        doc = self.embeddings(doc)
        query = query / (query.norm(p=2, dim=-1, keepdim=True) + 1e-16)
        doc = doc / (doc.norm(p=2, dim=-1, keepdim=True) + 1e-16)
        return torch.bmm(query, doc.transpose(-1, -2))
    
#         q = query.numpy().tolist()
#         d = doc.numpy().tolist()

#         a_emb, b_emb = torch.Tensor(self.embeddings.weight[q]), torch.Tensor(self.embeddings.weight[d])
#         norm_a = a_emb.norm(p=2, dim=1)
#         norm_b = b_emb.norm(p=2, dim=1)

#         sim = a_emb.matmul(b_emb.t())/norm_a/norm_b
#         matching_matrix = sim.nan_to_num()
#         return matching_matrix

    def _apply_kernels(self, matching_matrix: torch.FloatTensor) -> torch.FloatTensor:
        KM = []
        for kernel in self.kernels:
            # shape = [B]
            K = torch.log1p(kernel(matching_matrix).sum(dim=-1)).sum(dim=-1)
            KM.append(K)
                
        # shape = [B, K]
        kernels_out = torch.stack(KM, dim=1)
        return kernels_out

    def predict(self, inputs: Dict[str, torch.Tensor]) -> torch.FloatTensor:
        # shape = [Batch, Left, Embedding], [Batch, Right, Embedding]
        query, doc = inputs['query'], inputs['document']
        
        # shape = [Batch, Left, Right]
        matching_matrix = self._get_matching_matrix(query, doc)
        # shape = [Batch, Kernels]
        kernels_out = self._apply_kernels(matching_matrix)
        # shape = [Batch]
        out = self.mlp(kernels_out)
        return out

    
def collate_fn(batch_objs: List[List[int]], name: str = 'query'):
    max_len_q1 = -1

    is_triplets = False
    for elems in batch_objs:
        max_len_q1 = max(len(elems), max_len_q1)

    q1s = []

    for elems in batch_objs:
        pad_len1 = max_len_q1 - len(elems)

        q1s.append(elems + [0] * pad_len1)

    ret_left = {name: torch.LongTensor(q1s)}

    return ret_left

In [300]:
MLP_PATH = './pkl/mlp_pretr.tor'
EMB_PATH_KNRM = './pkl/pretr_emb_krnm_50.tor'
EMB_PATH_KNRM_TXT = './pkl/pretr_emb_krnm_50.txt'

VOCAB_PATH = './pkl/vocab'
VOCAB_PATH_J = './pkl/vocab.json'

In [294]:
# with open(VOCAB_PATH, "rb") as fin:
#     vocab = pickle.load(fin)

In [295]:
# with open(VOCAB_PATH_J, 'w') as fout:
#     json.dump(vocab,fout,indent=4)

In [296]:
with open(VOCAB_PATH_J, 'r') as fin:
    vocab = json.load(fin)

In [89]:
# with open(EMB_PATH_KNRM, "rb") as fin:
#     emb_matrix = pickle.load(fin)

In [23]:
# with open(EMB_PATH_KNRM_TXT, "w") as fout:
#     for line in emb_matrix:
#         str_arr = " ".join(["{:10.7f}".format(ele).strip() for ele in line])
#         fout.writelines(str_arr+"\n")

In [41]:
# with open(EMB_PATH_KNRM_TXT, "r") as fin:
#     emb_matrix = fin.readlines()
    
# emb_matrix = torch.Tensor([list(map(float,re.sub(r"\n", "", ele).split(" "))) 
#                            for ele in emb_matrix
#                           ]
#                          )

In [301]:
mlp = torch.load(MLP_PATH)
mlp.state_dict()

OrderedDict([('1.weight',
              tensor([[-1.3675e-03,  9.7939e-02, -1.5025e-01, -1.3421e-01, -6.9578e-02,
                        5.1119e-02,  1.1802e-03,  1.5375e-01, -2.2322e-03,  6.3431e-02,
                       -5.8350e-02, -9.0235e-02, -2.9208e-01, -2.8554e-01, -2.5503e-01,
                       -1.5083e-01, -3.1750e-02,  6.9409e-02, -1.1293e-01, -4.6203e-02,
                        1.0435e-01,  1.9258e-01,  7.3940e-03,  1.8652e-01,  2.4456e-02,
                        7.1679e-02,  2.0797e-01, -1.4159e-01, -1.0320e-01, -5.9219e-02],
                      [-7.1165e-02,  1.5776e-01, -1.1824e-01, -8.3621e-02, -1.2641e-01,
                       -1.6893e-01, -1.0427e-01,  1.5859e-01,  8.2196e-02,  8.7204e-02,
                       -6.6780e-04, -1.2414e-01, -1.4524e-02, -2.1770e-01, -1.7867e-01,
                       -1.4649e-01,  5.4514e-02,  4.6886e-02, -1.3491e-01, -5.8005e-02,
                        6.6114e-02,  1.3351e-01,  2.9996e-02, -1.1296e-02,  9.0604e-02,
     

In [377]:
emb_matrix = torch.load(EMB_PATH_KNRM)
knrm = KNRM(emb_matrix["weight"], freeze_embeddings=True)

In [378]:
emb_matrix['weight']

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0022, -0.0369,  0.0055,  ..., -0.1204, -0.0544, -0.0585],
        [ 0.4532,  0.0598, -0.1058,  ...,  0.5324, -0.2510,  0.6255],
        ...,
        [-0.0022, -0.0369,  0.0055,  ..., -0.1204, -0.0544, -0.0585],
        [-0.5138, -0.9359, -0.1946,  ...,  0.2532,  0.1363, -0.0305],
        [-0.3993, -0.5018, -0.8642,  ...,  0.4815, -0.2657, -0.2634]])

In [379]:
knrm.mlp = mlp
knrm.mlp.state_dict()

OrderedDict([('1.weight',
              tensor([[-1.3675e-03,  9.7939e-02, -1.5025e-01, -1.3421e-01, -6.9578e-02,
                        5.1119e-02,  1.1802e-03,  1.5375e-01, -2.2322e-03,  6.3431e-02,
                       -5.8350e-02, -9.0235e-02, -2.9208e-01, -2.8554e-01, -2.5503e-01,
                       -1.5083e-01, -3.1750e-02,  6.9409e-02, -1.1293e-01, -4.6203e-02,
                        1.0435e-01,  1.9258e-01,  7.3940e-03,  1.8652e-01,  2.4456e-02,
                        7.1679e-02,  2.0797e-01, -1.4159e-01, -1.0320e-01, -5.9219e-02],
                      [-7.1165e-02,  1.5776e-01, -1.1824e-01, -8.3621e-02, -1.2641e-01,
                       -1.6893e-01, -1.0427e-01,  1.5859e-01,  8.2196e-02,  8.7204e-02,
                       -6.6780e-04, -1.2414e-01, -1.4524e-02, -2.1770e-01, -1.7867e-01,
                       -1.4649e-01,  5.4514e-02,  4.6886e-02, -1.3491e-01, -5.8005e-02,
                        6.6114e-02,  1.3351e-01,  2.9996e-02, -1.1296e-02,  9.0604e-02,
     

In [380]:
def _read_glove_embeddings(file_path: str) -> Dict[str, List[str]]:
    res = dict([tuple(line.split(" ", 1)) for line in open(file_path, 'r')]) 
    fin = dict([(k, list(map(float,v.split(" ")))) for k,v in res.items() if k not in string.punctuation])
    return fin 

In [381]:
%%time
glove_emb = _read_glove_embeddings(EMB_PATH_GLOVE)

CPU times: user 7.09 s, sys: 1.06 s, total: 8.16 s
Wall time: 8.23 s


In [433]:
emb = glove_emb.get("the")

In [435]:
norm = emb / np.linalg.norm(emb, axis = 0, keepdims = True)

In [437]:
np.linalg.norm(norm)

1.0

## FAISS train

In [391]:
def get_idx_to_text_mapping(inp_df: pd.DataFrame) -> Dict[str, str]:
    left_dict = (
        inp_df
        [['id_left', 'text_left']]
        .drop_duplicates()
        .set_index('id_left')
        ['text_left']
        .to_dict()
    )
    right_dict = (
        inp_df
        [['id_right', 'text_right']]
        .drop_duplicates()
        .set_index('id_right')
        ['text_right']
        .to_dict()
    )
    left_dict.update(right_dict)
    return left_dict

def get_glue_df(partition_type: str) -> pd.DataFrame:
    assert partition_type in ['dev', 'train']
    glue_df = pd.read_csv(
        glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', dtype=object)
    glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
    glue_df_fin = pd.DataFrame({
        'id_left': glue_df['qid1'],
        'id_right': glue_df['qid2'],
        'text_left': glue_df['question1'],
        'text_right': glue_df['question2'],
        'label': glue_df['is_duplicate'].astype(int)
    })
    return glue_df_fin

def hadle_punctuation(inp_str: str) -> str:
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    out = regex.sub('', inp_str)
    return out

def simple_preproc(inp_str: str) -> List[str]:
    rem_puct_str = hadle_punctuation(inp_str).lower()       
    return nltk.word_tokenize(rem_puct_str)

def tokenized_text_to_index(tokenized_text: List[str]) -> List[int]:
    return [vocab.get(item, vocab['OOV']) for item in tokenized_text]

def avg_emb(toks):
    np.random.seed(17)
#     dim_of_embed = len(glove_emb.get('the'))
    zeros = np.zeros_like(glove_emb.get('the'))
    unk_embedding = emb_matrix["weight"][1].numpy() #np.random.uniform(-100, 100, dim_of_embed) 
    avg_emb = zeros #unk_embedding
    
    if len(toks) != 0:
        if toks[0] is None: return None
        for tok in toks:
            avg_emb += glove_emb.get(tok, unk_embedding)
        return avg_emb / len(toks) 
    else:
        return unk_embedding

## Query Preprocessing

In [66]:
df_tr = get_glue_df('train')
df_val = get_glue_df('dev')

In [67]:
idx_to_text_mapping = get_idx_to_text_mapping(df_tr)
idx_to_text_mapping_val = get_idx_to_text_mapping(df_val)

## Sampling

In [451]:
pos_sample  = np.random.choice(list(idx_to_text_mapping.keys()), 300000, replace=False)
documents = dict([(k, idx_to_text_mapping[k]) for k in pos_sample])

pos_sample_val  = np.random.choice(list(idx_to_text_mapping_val.keys()), 10, replace=False)
query = {"queries": [idx_to_text_mapping_val[k] for k in pos_sample_val]}

In [452]:
doc_dict = {"documents": documents}

In [453]:
query["queries"].extend(['关于板球有哪些令人兴奋的事实？','计算机科学工程、软件工程和机电一体化之间有什么区别？'])

In [422]:
query

{'queries': ['Who might President Donald Trump appoint to their cabinet?',
  'What is the difference between story writing and screenplay writing for movies?',
  'What are the pros and cons of Ceasing Rs. 500 and Rs.1000 Currency Notes in India?',
  'I have existing MR app that can run on YARN. Can I run this app on Tez on YARN without any modification? If not, will it be possible in future?',
  'Why does my cat sing and meow at night right when I go to bed?',
  'What is the structure of the cell membrane?',
  'How do I earn money through online in India?',
  'What are some good songs to make a texting lyric prank?',
  'Why is high fructose corn syrup so bad?',
  'How can I get more upvotes and views in Quora?',
  '关于板球有哪些令人兴奋的事实？',
  '计算机科学工程、软件工程和机电一体化之间有什么区别？']}

In [454]:
with open("./jl/query.json", "w") as f:
    json.dump(query,f, indent=2)
    
with open("./jl/documents.json", "w") as f:
    json.dump(doc_dict,f, indent=2)    

In [133]:
# sampled_emb     = np.array([(k, idx_to_text_mapping_samp[k]) for k in pos_sample],dtype=object)
# sampled_emb_val = np.array([(k, idx_to_text_mapping_val_samp[k]) for k in pos_sample_val],dtype=object)

## Tokenization

In [423]:
doc_tokens   = np.array([(q,simple_preproc(text)) for q,text in doc_dict["documents"].items()], dtype=object)
query_tokens = np.array([simple_preproc(text) for text in query['queries']], dtype=object)

In [424]:
%%time
doc_emb   = np.array([(q, avg_emb(toks)) for q,toks in doc_tokens], dtype=object)

lang_check = [True if detect(text)=='en' else False for text in query['queries']]
query_tokens = np.array([simple_preproc(text) if lang_check[i] == True else [None] 
                         for i, text in enumerate(query['queries'])], dtype=object)    
query_emb    = np.array([avg_emb(toks) for toks in query_tokens], dtype=object)
# query = query_emb #.astype('float32') 

CPU times: user 1.72 s, sys: 52 ms, total: 1.77 s
Wall time: 1.78 s


In [425]:
len(doc_emb)

20000

## Train FAISS

In [438]:
vectors = np.array([emb for _,emb in doc_emb], dtype=object).astype('float32')
len(vectors)

20000

In [441]:
norm = np.linalg.norm(vectors, axis = 1, keepdims = True)
norm_vec = vectors / norm

In [443]:
np.linalg.norm(norm_vec[0])

1.0

In [448]:
k = 1000
code_size = 5
dim = len(glove_emb.get('the'))
quantizer = faiss.IndexFlatIP(dim) 
index = faiss.IndexIVFFlat(quantiser, dim, k)
# index = faiss.IndexIVFPQ(quantizer, dim, k, code_size, 8)

In [449]:
# index = faiss.index_factory(dim, 'IVF%s,PQ5' % str(k))
print(index.is_trained)
index.train(norm_vec)  

False


In [450]:
print(index.is_trained)
print(index.ntotal)
index.add(norm_vec)
print(index.ntotal)

True
0
20000


In [447]:
index.ntotal

20000

## Search Neighbour

In [412]:
# query = query_emb.astype('float32')
len(query_emb)

12

In [413]:
np.expand_dims(query_emb[0],axis=0)

array([[ 0.3231325 ,  0.03252687, -0.13553125, -0.04481263,  0.35215922,
         0.24063563, -0.47955576,  0.07188312,  0.14682372, -0.07151325,
         0.36719375,  0.07929625, -0.04210988, -0.170715  ,  0.39910506,
         0.50338725,  0.38878975, -0.27713   , -0.08214625, -0.189309  ,
         0.24382862, -0.07910612,  0.05398825,  0.1336045 ,  0.24808225,
        -1.64630375, -0.12428875, -0.22289987,  0.225227  , -0.364803  ,
         3.0778125 ,  0.0458575 , -0.66569012, -0.50898875, -0.11025161,
        -0.40969789,  0.0360045 ,  0.15198763, -0.012987  , -0.06125725,
        -0.161864  , -0.18182025,  0.24411025, -0.06071225, -0.286624  ,
         0.14335325, -0.14408114,  0.03562075, -0.15558975, -0.0684179 ]])

In [414]:
i = 0

In [415]:
index.nprobe = 10
topn = 100
D, I = index.search(np.expand_dims(query_emb[i],axis=0).astype("float32"), topn) 
D, I

(array([[0.31689   , 0.7277256 , 0.78914136, 0.8043233 , 0.8053964 ,
         0.87429404, 0.8831511 , 0.9038441 , 0.9141702 , 0.9396117 ,
         0.9397359 , 0.9474556 , 0.9692233 , 0.99531806, 1.0009531 ,
         1.0317323 , 1.0401981 , 1.0459554 , 1.0464041 , 1.089001  ,
         1.0985188 , 1.1051904 , 1.1065682 , 1.1101832 , 1.1203438 ,
         1.1249537 , 1.1418786 , 1.1479276 , 1.1482043 , 1.1484969 ,
         1.1485527 , 1.1502395 , 1.151597  , 1.1549463 , 1.1687927 ,
         1.1733601 , 1.1758177 , 1.1784341 , 1.1859734 , 1.1899159 ,
         1.1908655 , 1.2026486 , 1.2038497 , 1.2078103 , 1.2127271 ,
         1.2155824 , 1.2184826 , 1.2184826 , 1.2223215 , 1.224473  ,
         1.2289991 , 1.231406  , 1.235086  , 1.237471  , 1.2408818 ,
         1.2414145 , 1.2504778 , 1.250543  , 1.25806   , 1.2597885 ,
         1.2612245 , 1.2642522 , 1.2655798 , 1.2746127 , 1.2747462 ,
         1.2767888 , 1.2781448 , 1.2786742 , 1.2787199 , 1.2791548 ,
         1.2823589 , 1.2868885 , 1

In [308]:
# indx = I[(I>-1)]
# toks_to_idx = [tokenized_text_to_index(tok) for _,tok in doc_tokens[indx]]
# padded_toks_to_idx = collate_fn(toks_to_idx, name = 'document')

# toks_to_idx = [tokenized_text_to_index(tok) for tok in np.expand_dims(query_tokens[i],axis=0)]*topn
# padded_toks_to_idx.update(collate_fn(toks_to_idx, name = 'query'))

# preds = knrm.predict(padded_toks_to_idx)
# ind_preds = preds.squeeze(1).sort(descending=True)[1].tolist()
# rel_doc_ids = [k for k, toks in doc_tokens[indx][ind_preds]]

# res = [tuple([k,text]) for k,text in doc_dict["documents"].items() if k in rel_doc_ids][:10]

In [327]:
fin_rel = []
lang_check = [True if detect(text)=='en' else False for text in query['queries']]
query_tokens = np.array([simple_preproc(text) if lang_check[i] == True else [None] 
                         for i, text in enumerate(query['queries'])], dtype=object)    
query_emb    = np.array([avg_emb(toks)  if len(toks) >= 2 else None for toks in query_tokens], dtype=object)
#         query = query_emb.astype('float32')   

for i,q in enumerate(query_emb):
    if q is not None:  
        print(i)
        index.nprobe = 5
        topn = 100 #index.ntotal
        D, I = index.search(np.expand_dims(q,axis=0).astype("float32"), topn)      
        indx = I[(I>-1)]
        toks_to_idx = [tokenized_text_to_index(tok) for _,tok in doc_tokens[indx]]
        padded_toks_to_idx = collate_fn(toks_to_idx, name = 'document')
    
        toks_to_idx = [tokenized_text_to_index(tok) for tok in np.expand_dims(query_tokens[i],axis=0)]*len(indx)
        padded_toks_to_idx.update(collate_fn(toks_to_idx, name = 'query'))

        print(padded_toks_to_idx["query"].shape, padded_toks_to_idx["document"].shape)
                
        preds = knrm.predict(padded_toks_to_idx)
        ind_preds = preds.squeeze(1).sort(descending=True)[1].tolist()
        rel_doc_ids = [k for k, toks in doc_tokens[indx][ind_preds]]

        res = [tuple([k,text]) for k,text in doc_dict["documents"].items() if k in rel_doc_ids][:10]
        fin_rel.append(res)
    else:
        fin_rel.append(None)

0
torch.Size([100, 14]) torch.Size([100, 59])
1
torch.Size([85, 12]) torch.Size([85, 28])
2
torch.Size([72, 9]) torch.Size([72, 30])
3
torch.Size([89, 13]) torch.Size([89, 40])
4
torch.Size([95, 20]) torch.Size([95, 37])
5
torch.Size([58, 7]) torch.Size([58, 24])
6
torch.Size([88, 8]) torch.Size([88, 30])
7
torch.Size([100, 12]) torch.Size([100, 57])
9
torch.Size([66, 20]) torch.Size([66, 37])


In [323]:
padded_toks_to_idx["document"].shape

torch.Size([85, 28])